In [1]:
import scipy.misc
import tensorflow as tf
import matplotlib.pyplot as plt

from nst_utils import load_vgg_model, reshape_and_normalize_image, generate_noise_image, save_image
from nst_app_utils import compute_content_cost, compute_style_cost, total_cost
from config import CONFIG

In [2]:
# Step 1: Create an interactive session
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [3]:
# Step 2: Load the content image
content_image = scipy.misc.imread(CONFIG.CONTENT_IMAGE)
content_image = reshape_and_normalize_image(content_image)

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [4]:
# Step 3: Load the style image
style_image = scipy.misc.imread(CONFIG.STYLE_IMAGE)
style_image = reshape_and_normalize_image(style_image)

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [5]:
# Step 4: Randomly initialize the image to be generated
generated_image = generate_noise_image(content_image)

In [6]:
# Step 5: Load the VGG16 model
model = load_vgg_model(CONFIG.VGG_MODEL)

In [7]:
# Step 6: Build the tensorflow graph
# Step 6a: Run the content image through VGG16 model and compute content cost
sess.run(model['input'].assign(content_image))
J_content = compute_content_cost(sess, model, CONFIG.CONTENT_LAYER)

# Step 6b: Run the style image through VGG16 model and compute style cost
sess.run(model['input'].assign(style_image))
J_style = compute_style_cost(sess, model, CONFIG.STYLE_LAYERS)

# Step 6c: Compute the total cost
J = total_cost(J_content, J_style, alpha = CONFIG.ALPHA, beta = CONFIG.BETA)

# Step 6d: Define the optimizer and learning rate
optimizer = tf.train.AdamOptimizer(CONFIG.LEARNING_RATE)
train_step = optimizer.minimize(J)

In [8]:
# Step 7: Run graph for a large number of iterations, updating the generated image at every step
# Initialize global variable
sess.run(tf.global_variables_initializer())

# Run the noisy initial generated image through the model.
sess.run(model['input'].assign(generated_image))

for i in range(CONFIG.NUM_ITERATIONS):

    # Run the session on the train_step to minimize the total cost
    sess.run(train_step)

    # Compute the generated image by running the session on the current model['input']
    generated_image = sess.run(model['input'])

    # Print every 20 iteration.
    if i % 20 == 0:
        Jt, Jc, Js = sess.run([J, J_content, J_style])
        print("Iteration " + str(i) + " :")
        print("total cost = " + str(Jt))
        print("content cost = " + str(Jc))
        print("style cost = " + str(Js))

        # save current generated image in the "/output" directory
        save_image("output/" + str(i) + ".png", generated_image)

# save last generated image

save_image('output/generated_image.jpg', generated_image)

Iteration 0 :
total cost = 5054266400.0
content cost = 7856.694
style cost = 126354696.0
Iteration 20 :
total cost = 940689860.0
content cost = 15193.847
style cost = 23513448.0
Iteration 40 :
total cost = 483279650.0
content cost = 16716.746
style cost = 12077812.0
Iteration 60 :
total cost = 314046500.0
content cost = 17374.97
style cost = 7846818.5
Iteration 80 :
total cost = 231551940.0
content cost = 17730.758
style cost = 5784365.5
Iteration 100 :
total cost = 184808030.0
content cost = 17992.84
style cost = 4615702.5
Iteration 120 :
total cost = 154239920.0
content cost = 18213.88
style cost = 3851444.5
Iteration 140 :
total cost = 132175140.0
content cost = 18364.883
style cost = 3299787.2
Iteration 160 :
total cost = 115032570.0
content cost = 18493.506
style cost = 2871190.8
Iteration 180 :
total cost = 101095310.0
content cost = 18591.777
style cost = 2522734.8
